In [90]:
import numpy as np
from scipy.sparse import coo_matrix
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical
from QLS.hhl import HHL              # quantum HHL
import time

In [91]:
# ----- user parameters ------------------------------------------------------
make_hermitian = True       # True → Hermitian SPD; False → general non-Hermitian
target_kappa   = 1e1        # desired condition number κ(A)
density        = 0.8        # fraction of nonzero entries (0<density≤1)
noise_level    = 1e1       # relative off-diag noise for SPD case

In [92]:
# ----- problem definition ---------------------------------------------------
NUM_WORK_QUBITS = 3                
DIM             = 2 ** NUM_WORK_QUBITS

In [93]:
# ----- helpers ---------------------------------------------------------------
def generate_sparse_spd(n, kappa, density, noise_level):
    """Hermitian SPD with log-spaced eigenvalues and random sparse off-diag noise."""
    # 1) log-spaced eigenvalues
    eigs = np.logspace(0, np.log10(kappa), n)
    # 2) diagonal entries
    rows = np.arange(n); cols = rows; data = eigs
    A = coo_matrix((data, (rows, cols)), shape=(n, n))
    # 3) add symmetric off-diagonal noise
    total = n*n
    nnz   = int(density*total)
    off   = max(nnz - n, 0)
    off  -= off % 2
    half  = off//2
    if half>0:
        i = np.random.randint(0,n,half*2)
        j = np.random.randint(0,n,half*2)
        mask = (i!=j)
        i,j = i[mask][:half], j[mask][:half]
        eps  = noise_level * eigs.min()
        vals = np.random.uniform(-eps, eps, size=half)
        A   = A + coo_matrix((vals,(i,j)),shape=(n,n))\
                + coo_matrix((vals,(j,i)),shape=(n,n))
    return A.tocsr()

In [94]:
def generate_general(n, kappa, density):
    """General (non-Hermitian) matrix with approx κ via SVD, then sparsified."""
    # 1) U, V random orthonormal
    U,_  = np.linalg.qr(np.random.randn(n,n))
    V,_  = np.linalg.qr(np.random.randn(n,n))
    # 2) singular values
    s    = np.logspace(0, np.log10(kappa), n)
    A    = U @ np.diag(s) @ V.T
    # 3) sparsify by zeroing random entries
    mask = (np.random.rand(n,n) < density)
    A   *= mask
    return A

In [95]:
# ----- build A ----------------------------------------------------------------
if make_hermitian:
    A_sparse = generate_sparse_spd(DIM, target_kappa, density, noise_level)
    A        = A_sparse.toarray()
else:
    A = generate_general(DIM, target_kappa, density)


In [96]:
# ----- checks & print ---------------------------------------------------------
is_herm = np.allclose(A, A.conj().T, atol=1e-12)
cond_A  = np.linalg.cond(A)
nnz     = np.count_nonzero(A)
print(f"A (dim={DIM}×{DIM}), Hermitian? {is_herm}, κ(A) ≈ {cond_A:.3e}, "
      f"sparsity={nnz}/{DIM*DIM}={nnz/(DIM*DIM):.2%}\n")
print("A =\n", A)   # uncomment to see the full matrix

A (dim=8×8), Hermitian? True, κ(A) ≈ 3.440e+02, sparsity=42/64=65.62%

A =
 [[  1.           0.           0.          -1.17677793  -7.16330063
   -3.42789637   5.90555989   7.08407339]
 [  0.           1.38949549  -3.06239958   0.           3.25116496
    4.7382878   -0.31346685   0.        ]
 [  0.          -3.06239958   1.93069773  -5.32152354 -13.26619203
   -4.3676361    0.           0.        ]
 [ -1.17677793   0.          -5.32152354   2.6826958   -3.42677156
    1.07295139   0.           0.        ]
 [ -7.16330063   3.25116496 -13.26619203  -3.42677156   3.72759372
    0.           7.64345956   0.        ]
 [ -3.42789637   4.7382878   -4.3676361    1.07295139   0.
    5.17947468  -8.6127288    0.        ]
 [  5.90555989  -0.31346685   0.           0.           7.64345956
   -8.6127288    7.19685673   7.35964008]
 [  7.08407339   0.           0.           0.           0.
    0.           7.35964008  10.        ]]


In [97]:
# ----- right-hand side -------------------------------------------------------
b_vec = np.zeros(DIM, dtype=complex if np.iscomplexobj(A) else float)
b_vec[0] = 1
b_vec

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [98]:
# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)
)

In [99]:
# ----- quantum (HHL) solution ------------------------------------------------
# calculate the time for the quantum solver
start_time = time.time()
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    sv           = Statevector(result.state).data
    total_qubits = int(np.log2(len(sv)))
    base_index   = 1 << (total_qubits - 1)
    amps         = np.array([sv[base_index + i]
                             for i in range(2 ** n_work_qubits)])
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)
# calculate the time taken by the quantum solver
end_time = time.time()
# print time taken
print(f"Quantum solver took {end_time - start_time:.3f} seconds.")

Quantum solver took 44.779 seconds.


In [100]:
# ----- results ---------------------------------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum, "\n")
print("Classical Euclidean norm:", classical_res.euclidean_norm)
print("Quantum   Euclidean norm:", quantum_res.euclidean_norm, "\n")
print("‖x_classical − x_quantum‖₂ =",
      np.linalg.norm(x_classical - x_quantum))

Classical solution vector: [-2.47172127 -4.37888619 -0.05266435 -0.23594717  0.9174287   0.54790127
 -1.098481    2.55942797]
Quantum   solution vector: [-2.47608548-7.43038659e-11j -4.38558862-1.31456391e-10j
 -0.05241424-1.54602672e-12j -0.23608149-7.03681069e-12j
  0.91886258+2.75229900e-11j  0.54877782+1.64620788e-11j
 -1.10006146-3.29662666e-11j  2.56365009+7.69037458e-11j] 

Classical Euclidean norm: 5.851640640438257
Quantum   Euclidean norm: 5.860953087954181 

‖x_classical − x_quantum‖₂ = 0.009337984183610912
